In [55]:
import pandas as pd
import resource 
import pickle as pickle
import parquet
import pyarrow
import feather
import json
import numpy as np

%load_ext memory_profiler

file = 'block_total.csv'
total_lines = 17_703_908

#LCLid,tstp,energy
pred = (lambda x: (x['energy'].astype('float')) < 0.3)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [60]:
%%time

df = pd.read_csv(file)
# feather.write_dataframe(df, "all.p")
# for c in df:
#     pickle.dump(df.loc[:, [c]], open( str(c) + ".p", "wb" ) )
#     feather.write_dataframe(df.loc[:, [c]],  str(c) + ".p")


CPU times: user 8.49 s, sys: 1.33 s, total: 9.82 s
Wall time: 12.4 s


In [93]:
# len(df['tstp'].unique())
df['LCLid']= df['LCLid'].astype('category')
df['tstp']= df['tstp'].astype('category')
# df['energy'] = df['energy'] / 10
df['energy'] = df['energy'].astype('int16')
# df[lambda x: (x['energy'].astype('int16')) - x['energy'] > 10e-15]


In [95]:
df.memory_usage(deep=True).sum()
# df

146369798

In [ ]:
## %%time
%%memit

# col = pickle.load( open( "energy.p", "rb" ) )
col = feather.read_dataframe("energy.p")

indices = col[pred].index.tolist()

df = pd.read_csv(file, skiprows=indices)

print(df.shape)


In [23]:
%%time
%%memit

df = pd.read_csv(file)

indices = df[pred].index

df.drop(indices, inplace=True)
print(df.shape)


(3245562, 3)
peak memory: 2955.27 MiB, increment: 1155.29 MiB
CPU times: user 12.3 s, sys: 2.08 s, total: 14.4 s
Wall time: 14.9 s


In [63]:
%%time
%%memit

col1 = pickle.load( open("LCLid.p", "rb" ))
col2 = pickle.load( open("tstp.p", "rb" ))
col3 = pickle.load( open("energy.p", "rb" ))

cols = pd.concat([col1, col2, col3], axis=1)
indices = cols[pred].index
cols.drop(indices, inplace=True)
print(cols.shape)


(3245562, 3)
peak memory: 2944.52 MiB, increment: 382.06 MiB
CPU times: user 7.05 s, sys: 2.54 s, total: 9.59 s
Wall time: 10.5 s
